In [1]:
!sudo apt-get install libmetis-dev
!pip install metis
import metis
import random
import tensorflow as tf
import numpy as np
from scipy import sparse
import scipy.io as sio

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libmetis-dev is already the newest version (5.1.0.dfsg-5).
0 upgraded, 0 newly installed, 0 to remove and 30 not upgraded.


In [2]:
# hyperparameters
hidden = 512 # number of hidden units in the encoder layer
latent = 256 # dimension of the latent variables
learning_rate = 0.01
epochs = 200
nparts = 1500 # number of partitions
batch_size = 20 # number of clusters per batch
K = 3 # number of Lanczos iterations

In [3]:
filename = '/content/drive/MyDrive/GRAPH DATA/reddit.mat' # dataset

In [4]:
mat_dict = sio.loadmat(filename)
A = mat_dict['A'].ceil()
X = mat_dict['X']
Y = mat_dict['Y']
train_mask = mat_dict['train_mask'].squeeze().astype(bool)
val_mask = mat_dict['val_mask'].squeeze().astype(bool)
test_mask = mat_dict['test_mask'].squeeze().astype(bool)

In [5]:
def cluster_graph(A, nparts):
  if nparts == 1:
    edge_cuts, parts = 0, [0, ] * A.shape[0]
  else:
    edge_cuts, parts = metis.part_graph([neighbors for neighbors in A.tolil().rows], nparts=nparts)
  print('Number of edge cuts: %d.' % edge_cuts)
  cluster_dict = {}
  for index, part in enumerate(parts):
    if part not in cluster_dict:
      cluster_dict[part] = []
    cluster_dict[part].append(index)
  return cluster_dict

# the clustering algorithm (METIS)
cluster_dict = cluster_graph(A, nparts)

Number of edge cuts: 9609639.


In [6]:
def preprocess_support(A):
  N = A.shape[1]
  D = sparse.csr_matrix(A.sum(axis=1))
  norm = D.power(-0.5)
  L = sparse.eye(N, dtype='float32') - A.multiply(norm).T.multiply(norm)
  max_eigval = sparse.linalg.eigsh(L, k=1, return_eigenvectors=False)[0]
  L_ = 2.0 / max_eigval * L - sparse.eye(N, dtype='float32')
  return L_

def toTensorSparse(S):
  return tf.constant(S.todense())

def toTensor(T):
  return tf.constant(T)

In [7]:
# layer classes

class bilinear_layer:

  def __init__(self, indim, outdim):
    pass

  def __call__(self, tensor):
    return tf.linalg.matmul(tensor, tf.transpose(tensor))

# unused
class FC_layer:

  def __init__(self, indim, outdim):
    initial_value = tf.initializers.he_normal()((indim, outdim,))
    self.weight = tf.Variable(initial_value=initial_value, trainable=True)

  def __call__(self, tensor):
    return tf.linalg.matmul(tensor, self.weight)

class GC_layer:

  def __init__(self, indim, outdim):
    global K
    self.K = K + 1
    delta = np.zeros((outdim, K + 1, K + 1), dtype='float32')
    for o in range(outdim):
      delta[o, 0, 0] = 1.0
    self.filter = tf.Variable(initial_value=delta, trainable=True)
    initial_value = tf.initializers.he_normal()((indim, outdim,))
    self.weight = tf.Variable(initial_value=initial_value, trainable=True)

  # Lanczos algorithm implemented for multiple vectors
  def Lanczos_algorithm(self, tensor, support, K, embed=False):
    Q = [tf.zeros(tensor.shape), tensor / tf.linalg.norm(tensor, axis=0)]
    C = [tf.zeros(tensor.shape[1])]
    B = [tf.zeros(tensor.shape[1])]
    for k in range(1, K + 1):
      if embed: # numpy pipeline
        z = tf.constant(support.dot(Q[k].numpy()))
      else: # tensorflow pipeline
        z = tf.linalg.matmul(support, Q[k])
      C.append(tf.einsum('ab, ab -> b', Q[k], z))
      z = z - Q[k] * C[k] - Q[k-1] * B[k-1]
      B.append(tf.linalg.norm(z, axis=0))
      Q.append(z / B[k])
    Vs = tf.transpose(tf.stack(Q[1:-1])) # Lanczos vectors
    Cs = tf.transpose(tf.stack(C[1:])) # diagonal Lanczos scalars
    Bs = tf.transpose(tf.stack(B[1:-1])) # off-diagonal Lanczos scalars
    Hs = tf.linalg.diag(Bs, k=-1) + tf.linalg.diag(Cs) + tf.linalg.diag(Bs, k=1) # tridiagonal matrix
    return Vs, Hs

  def __call__(self, tensor, support, embed=False):
    # see the following reference:
    # "Susnjara, A., Perraudin, N., Kressner, D., & Vandergheynst, P. (2015).
    # Accelerated filtering on graphs using lanczos method. arXiv preprint arXiv:1509.04537."
    tensor = tf.linalg.matmul(tensor, self.weight)
    V, H = self.Lanczos_algorithm(tensor, support, self.K, embed)
    delta = tf.one_hot(tf.zeros(tensor.shape[1], dtype=tf.uint8), self.K)
    norm = tf.linalg.norm(tensor, axis=0)
    eigvals, eigvecs = tf.linalg.eigh(H)
    T = tf.einsum('abc, acd, ade -> abe', eigvecs, self.filter, eigvecs)
    result = tf.einsum('abc, acd, ad, a -> ba', V, T, delta, norm)
    return result


In [8]:
# our model class (for the paper "Scalable Graph Variational Autoencoders")

class Model:

  def __init__(self, size_tuple, optimizer, nonlinear):
    self.sources = [] # variables to optimize
    self.build(size_tuple) # builds the model by stacking layers on each other
    self.optimizer = optimizer
    self.nonlinear = nonlinear
    self.Z_mean = None # mean embedding layer
    self.Z_var = None # variance embedding layer
    self.noise = None # the noise sample
    self.sample = None # self.Z_mean + self.Z_var * self.noise
    self.A_gamma = None # the reconstructions
  
  def build(self, size_tuple):
    X_dim, hidden, latent = size_tuple
    self.enc_layer = GC_layer(X_dim, hidden)
    self.enc_mean_layer = GC_layer(hidden, latent)
    self.enc_var_layer = GC_layer(hidden, latent)
    self.A_dec_gamma_layer = bilinear_layer(latent, latent)
    # filling the source array with weights
    layers = [self.enc_layer, self.enc_mean_layer, self.enc_var_layer]
    for layer in layers:
      self.sources.append(layer.weight)
      self.sources.append(layer.filter)
  
  # forward propagation in the encoder
  def encode(self, X, S):
    enc = self.nonlinear(self.enc_layer(X, S))
    enc_mean = self.enc_mean_layer(enc, S)
    enc_var = tf.math.exp(self.enc_var_layer(enc, S))
    return enc_mean, enc_var

  # returns only the node embeddings
  def embed(self, X, S):
    enc = self.nonlinear(self.enc_layer(X, S, embed=True))
    enc_mean = self.enc_mean_layer(enc, S, embed=True)
    return enc_mean

  # forward propagation in the decoder
  def decode(self, sample):
    A_dec_gamma = self.A_dec_gamma_layer(sample)
    return A_dec_gamma

  def predict(self, X, S):
    self.Z_mean, self.Z_var = self.encode(X, S)
    self.noise = tf.random.normal(self.Z_var.shape)
    self.sample = self.Z_mean + self.Z_var * self.noise # reparameterization trick
    self.A_gamma = self.decode(self.sample)

  def train(self, X, A, cluster_dict, batch_size, epochs):
    for epoch in range(epochs):
      # only a subgraph is used in the training process
      samples = random.sample(cluster_dict.keys(), batch_size)
      nodes = sum([cluster_dict[sample] for sample in samples], [])
      S_batch = toTensorSparse(preprocess_support(A[nodes].T[nodes]))
      A_batch = toTensor(A.T[nodes].T[nodes].todense())
      X_batch = tf.math.l2_normalize(toTensor(X[nodes]), axis=1)
      # optimization
      with tf.GradientTape() as tape:
        self.predict(X_batch, S_batch)
        losses = self.loss(A_batch, X_batch)
        loss_ = tf.reduce_sum(losses)
      print(epoch, [loss.numpy() for loss in losses], loss_.numpy())
      grads = tape.gradient(loss_, self.sources)
      self.optimizer.apply_gradients(zip(grads, self.sources))

  # Kullback–Leibler divergence
  def KL_Divergence(self):
    loss = 0.5 * tf.reduce_mean(self.Z_mean**2.0 + self.Z_var**2.0 - 2.0 * tf.math.log(self.Z_var) - 1.0)
    return loss

  # reconstruction loss
  def re_A_loss(self, A):
    density = tf.reduce_sum(A) / tf.size(A, out_type=tf.float32)
    pos_weight = (1.0 - density) / density
    loss = -0.5 * tf.reduce_mean(1.0 / (1.0 - density) * tf.nn.weighted_cross_entropy_with_logits(labels=A, logits=self.A_gamma, pos_weight=pos_weight))
    return -loss

  # list of all loss functions
  def loss(self, A, X):
    return self.KL_Divergence(), self.re_A_loss(A)


In [9]:
size_tuple = (X.shape[1], hidden, latent)
optimizer = tf.optimizers.Adam(learning_rate=learning_rate)
nonlinear = tf.nn.relu

model = Model(size_tuple, optimizer, nonlinear)

print('Training...')
model.train(X, A, cluster_dict, batch_size, epochs)


Training...
0 [0.003802459, 6.483147] 6.4869494
1 [0.025074247, 5.100501] 5.1255755
2 [0.07272147, 3.6049988] 3.6777203
3 [0.20927277, 2.2150311] 2.424304
4 [0.45730138, 1.592591] 2.0498924
5 [0.7325524, 1.001307] 1.7338594
6 [1.0259452, 1.1978085] 2.2237537
7 [1.1405663, 0.82289326] 1.9634596
8 [1.2961732, 1.0005741] 2.2967472
9 [1.1595438, 0.74875605] 1.9082998
10 [1.1969844, 0.9330629] 2.1300473
11 [0.9596665, 0.7973858] 1.7570523
12 [0.8047619, 0.8703625] 1.6751244
13 [0.6495329, 0.9633046] 1.6128376
14 [0.54674673, 1.1267725] 1.6735193
15 [0.46032926, 1.2836887] 1.744018
16 [0.4250397, 1.3477314] 1.7727711
17 [0.4143241, 1.3658344] 1.7801585
18 [0.4214787, 1.3453584] 1.7668371
19 [0.46001872, 1.2531964] 1.7132151
20 [0.5244157, 1.1124256] 1.6368413
21 [0.65035945, 0.9871469] 1.6375064
22 [0.7130035, 0.8845272] 1.5975307
23 [0.7841942, 0.8319147] 1.6161089
24 [0.85805744, 0.75830257] 1.61636
25 [0.8110677, 0.8012322] 1.6122999
26 [0.7944675, 0.86402565] 1.6584932
27 [0.80353653, 0.

In [10]:
S = preprocess_support(A)
X = tf.math.l2_normalize(toTensor(X), axis=1)
embs = model.embed(X, S) # node embeddings

In [11]:
# node clustering using the KMeans algorithm
from sklearn.cluster import KMeans
y_pred = KMeans(n_clusters=Y.shape[1]).fit(embs).predict(embs)
y_true = np.argmax(Y, axis=1)

In [12]:
# result
from sklearn.metrics import adjusted_mutual_info_score
print(adjusted_mutual_info_score(y_true, y_pred))

0.48847694277422654
